In [1]:
%reload_ext autoreload
%autoreload 2

import os
from pathlib import Path
import multiprocessing

import jax
import jax.numpy as jnp
import pandas as pd
import numpy as np
import numpyro
import numpyro.distributions as dist

from hbmep.config import Config
from hbmep.model import Baseline
from hbmep.model.utils import Site as site
from hbmep.utils.constants import RECTIFIED_LOGISTIC

PLATFORM = "cpu"
jax.config.update("jax_platforms", PLATFORM)
numpyro.set_platform(PLATFORM)

cpu_count = multiprocessing.cpu_count() - 2
numpyro.set_host_device_count(cpu_count)
numpyro.enable_x64()


#### Load config

In [2]:
root_path = Path(os.getcwd()).parent.parent.parent.absolute()
toml_path = os.path.join(root_path, "configs/J_RCML_000.toml")

config = Config(toml_path=toml_path)

config.BUILD_DIR = "/home/vishu/repos/hbmep-paper/reports/J_RCML_000/fix-2/student/inv"


2023-07-19 11:24:26,661 - hbmep.config - INFO - Verifying configuration ...
2023-07-19 11:24:26,661 - hbmep.config - INFO - Success!


In [3]:
class RectifiedLogistic(Baseline):
    def __init__(self, config: Config):
        super(RectifiedLogistic, self).__init__(config=config)
        self.link = RECTIFIED_LOGISTIC

    def _model(self, subject, features, intensity, response_obs=None):
        intensity = intensity.reshape(-1, 1)
        intensity = np.tile(intensity, (1, self.n_response))

        feature0 = features[0].reshape(-1,)

        n_data = intensity.shape[0]
        n_subject = np.unique(subject).shape[0]
        n_feature0 = np.unique(feature0).shape[0]

        with numpyro.plate(site.n_response, self.n_response, dim=-1):
            with numpyro.plate(site.n_subject, n_subject, dim=-2):
                """ Hyper-priors """
                mu_a = numpyro.sample(
                    site.mu_a,
                    dist.TruncatedNormal(150, 20, low=0)
                )
                sigma_a = numpyro.sample(site.sigma_a, dist.HalfCauchy(10))

                sigma_b = numpyro.sample(site.sigma_b, dist.HalfNormal(0.5))

                sigma_L = numpyro.sample(site.sigma_L, dist.HalfNormal(0.05))
                sigma_H = numpyro.sample(site.sigma_H, dist.HalfNormal(5))
                sigma_v = numpyro.sample(site.sigma_v, dist.HalfNormal(5))

                with numpyro.plate("n_feature0", n_feature0, dim=-3):
                    """ Priors """
                    a = numpyro.sample(
                        site.a,
                        dist.TruncatedNormal(mu_a, sigma_a, low=0)
                    )
                    b = numpyro.sample(site.b, dist.HalfNormal(sigma_b))

                    L = numpyro.sample(site.L, dist.HalfNormal(sigma_L))
                    H = numpyro.sample(site.H, dist.HalfNormal(sigma_H))
                    v = numpyro.sample(site.v, dist.HalfNormal(sigma_v))

                    # p = numpyro.sample(site.p, dist.HalfCauchy(10))

                    sigma_1 = numpyro.sample(
                        "sigma_1", dist.HalfCauchy(10)
                    )
                    sigma_2 = numpyro.sample(
                        "sigma_2", dist.HalfCauchy(10)
                    )

                    d_1 = numpyro.sample(
                        "d_1", dist.HalfCauchy(10)
                    )
                    d_2 = numpyro.sample(
                        "d_2", dist.HalfCauchy(10)
                    )

        """ Model """
        mu = numpyro.deterministic(
            site.mu,
            L[feature0, subject]
            + jnp.maximum(
                0,
                -1
                + (H[feature0, subject] + 1)
                / jnp.power(
                    1
                    + (jnp.power(1 + H[feature0, subject], v[feature0, subject]) - 1)
                    * jnp.exp(-b[feature0, subject] * (intensity - a[feature0, subject])),
                    1 / v[feature0, subject]
                )
            )
        )
        sigma = numpyro.deterministic(
            "sigma",
            sigma_1[feature0, subject]
            + sigma_2[feature0, subject] * mu
        )
        d = numpyro.deterministic(
            "d",
            2 + d_1[feature0, subject]
            + d_2[feature0, subject] * mu
        )

        with numpyro.plate(site.data, n_data):
            return numpyro.sample(
                site.obs,
                dist.StudentT(df=d, loc=mu, scale=sigma).to_event(1),
                obs=response_obs
            )



model = RectifiedLogistic(config=config)

In [4]:
df = pd.read_csv(model.csv_path)
print(df.compound_position.unique())

['C5M-C5L' 'C5M-C6L' 'C7L-C8L' '-C5M' 'C7M-C8L' 'C8M-C8L' '-C8M' '-C6L'
 '-C7M' '-C8L' 'C7M-C8M' 'C5L-C6L' '-C5L' 'C6L-C7L' '-C7L' 'C7M-C7L'
 'C6M-C7M' 'C5M-C6M' '-C6M' 'C6M-C7L' 'C6M-C6L']


In [5]:


""" Filter """
# ind = df.compound_position.isin(["-C5M"])
sub = [
    # ("amap06", "-C7L"),
    # ("amap06", "-C8L"),
    # ("amap08", "C6M-C7L"),
    ("amap01", "C5M-C5L"),
    ("amap01", "C5M-C6L"),
    ("amap01", "C7L-C8L"),
    ("amap01", "-C5M"),
    ("amap01", "C7M-C8L"),
    ("amap01", "C8M-C8L"),
    ("amap01", "-C8M"),
    ("amap01", "-C6L"),
    ("amap01", "-C7M"),
    ("amap01", "-C8L"),
    ("amap01", "C7M-C8M"),
    ("amap01", "C5L-C6L"),
]
ind = df[model.combination_columns].apply(tuple, axis=1).isin(sub)
# ind = df[model.subject].isin(["amap01"])

df = df[ind].copy()
df.reset_index(drop=True, inplace=True)

df, encoder_dict = model.load(df=df)

df.shape

2023-07-19 11:24:26,735 - hbmep.dataset.core - INFO - Artefacts will be stored here - /home/vishu/repos/hbmep-paper/reports/J_RCML_000/fix-2/student/inv
2023-07-19 11:24:26,735 - hbmep.dataset.core - INFO - Copied config to /home/vishu/repos/hbmep-paper/reports/J_RCML_000/fix-2/student/inv
2023-07-19 11:24:26,736 - hbmep.dataset.core - INFO - Processing data ...
2023-07-19 11:24:26,737 - hbmep.utils.utils - INFO - func:load took: 0.00 sec


(632, 43)

In [6]:
mcmc, posterior_samples = model.run_inference(df=df)


2023-07-19 11:24:26,756 - hbmep.model.baseline - INFO - Running inference with rectified_logistic ...


  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

2023-07-19 11:56:09,577 - hbmep.utils.utils - INFO - func:run_inference took: 31 min and 42.82 sec


In [7]:
mcmc.print_summary(prob=.95)



                     mean       std    median      2.5%     97.5%     n_eff     r_hat
       H[0,0,0]      2.03      0.75      1.86      0.96      3.50   8276.90      1.00
       H[0,0,1]      1.85      1.57      1.40      0.32      4.65   7452.28      1.00
       H[0,0,2]      0.84      0.33      0.75      0.47      1.43   5606.83      1.00
       H[1,0,0]      1.02      0.10      1.00      0.82      1.21  10188.03      1.00
       H[1,0,1]      1.91      1.49      1.47      0.51      4.58   6578.78      1.00
       H[1,0,2]      0.46      0.04      0.45      0.39      0.53   8645.52      1.00
       H[2,0,0]      1.06      0.80      0.82      0.17      2.65  13351.25      1.00
       H[2,0,1]      1.43      1.55      0.97      0.04      4.26   8500.47      1.00
       H[2,0,2]      0.43      0.53      0.23      0.01      1.47  11157.90      1.00
       H[3,0,0]      0.95      0.30      0.87      0.57      1.51   6777.22      1.00
       H[3,0,1]      1.78      1.50      1.34      0.

In [8]:
model.render_recruitment_curves(df=df, encoder_dict=encoder_dict, posterior_samples=posterior_samples)


2023-07-19 11:56:10,409 - hbmep.model.baseline - INFO - Rendering recruitment curves ...
2023-07-19 11:56:56,923 - hbmep.model.baseline - INFO - Saved to /home/vishu/repos/hbmep-paper/reports/J_RCML_000/fix-2/student/inv/recruitment_curves.pdf
2023-07-19 11:56:56,923 - hbmep.utils.utils - INFO - func:render_recruitment_curves took: 46.52 sec


In [9]:
model.render_predictive_check(df=df, encoder_dict=encoder_dict, posterior_samples=posterior_samples)


2023-07-19 11:56:57,014 - hbmep.model.baseline - INFO - Rendering Posterior Predictive Check ...


2023-07-19 11:57:53,295 - hbmep.model.baseline - INFO - Saved to /home/vishu/repos/hbmep-paper/reports/J_RCML_000/fix-2/student/inv/posterior_predictive_check.pdf
2023-07-19 11:57:53,299 - hbmep.utils.utils - INFO - func:render_predictive_check took: 56.29 sec


In [10]:
model.save(mcmc=mcmc)

2023-07-19 11:59:01,216 - hbmep.model.baseline - INFO - Saving inference data ...
2023-07-19 11:59:26,093 - hbmep.model.baseline - INFO - Saved to /home/vishu/repos/hbmep-paper/reports/J_RCML_000/fix-2/student/inv/mcmc.nc
2023-07-19 11:59:26,094 - hbmep.model.baseline - INFO - Rendering convergence diagnostics ...
2023-07-19 12:00:15,408 - hbmep.model.baseline - INFO - Saved to /home/vishu/repos/hbmep-paper/reports/J_RCML_000/fix-2/student/inv/diagnostics.csv
2023-07-19 12:00:15,409 - hbmep.model.baseline - INFO - Evaluating model ...
/home/vishu/repos/hbmep-paper/.venv/lib/python3.11/site-packages/arviz/stats/stats.py:803: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.7 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observati